In [4]:
import TableParser
import pandas as pd
import tempfile, zipfile
import os
import numpy as np

In [66]:
class TestTableParser(TableParser.TableParserItem):
    def __init__(self):
        pass
    
    def parse(self, series: pd.Series):
        mistake = {
            '0:00':'',
            'Jan':'01','Feb': '02','Mar':'03','Apr':'04','May':'05','Jun':'06','Jul':'07','Aug':'08','Sept':'09','Oct':'10','Nov':'11','Dec':'12',
            'o': '0','l': '1','b': '6','g': '9','q': '9','t': '7','v': '5','G': '6','F': '7','Z': '2','Q': '2','B': '8','O': '0','D': '0','E': '3','A': '4','S': '5',
            r'[ -/]|[:-@]|[\[-_]|[{-~]': '-',
        }
        for key, val in mistake.items():
            series = series.str.strip().str.replace(key, val, regex=True)
        return (pd.to_datetime(series, errors="ignore"), [])

    def fix_xlsx(self, in_file):
        tmpfd, tmp = tempfile.mkstemp(dir=os.path.dirname(in_file))
        os.close(tmpfd)
        filename = '[Content_Types].xml'
        data = ''
        with zipfile.ZipFile(in_file, 'r') as zin:
            with zipfile.ZipFile(tmp, 'w') as zout:
                for item in zin.infolist():
                    if item.filename != filename:
                        zout.writestr(item, zin.read(item.filename))
                    else:
                        data = zin.read(filename).decode()
        os.remove(in_file)
        os.rename(tmp, in_file)
        data = data.replace('/xl/sharedStrings.xml', '/xl/SharedStrings.xml')
        with zipfile.ZipFile(in_file, mode='a', compression=zipfile.ZIP_DEFLATED) as zf:
            zf.writestr(filename, data)

    ###############################################
    # to find the x,y coordinate of row with date #
    def find_break(self,df,data_row_num,data_col_num,main_axis):
        count_breaks = 0
        break_coordinate = 0
        if main_axis == "Y axis":
            date = df.iloc[:,data_col_num]
            date = date.isnull()
            for i in range(len(date)):
                if date[i]:
                    count_breaks +=1
                else: count_breaks = 0
                if count_breaks == 5:
                    break_coordinate = i 
                    break
        return break_coordinate
        
                
    def find_coordinate(self, x, y,df): # and defind axis
        test_list =[[0,0]]
        count_X_axis,count_Y_axis,main_axis = 0,0,"Y axis"
        
        for i in range(df.shape[0]):
            if len(test_list) == 11: 
                break
            for j in range(df.shape[1]):
                if  type(str(df.iloc[i,j])) == type("string") and len(str(df.iloc[i,j])) >=10:
                    if type(testTableParser.parse(pd.Series([str(df.iloc[i,j])]))[0][0]) == type(pd.to_datetime('31.01.2002')):
                        test_list.append([i,j])
                        if test_list[-1][1] > test_list[-2][1]:
                            count_X_axis += 1
                        if test_list[-1][0] > test_list[-2][0]:
                            count_Y_axis += 1  
                        if len(test_list) == 11:
                            break
#         print(test_list)
        if count_X_axis > count_Y_axis:
            main_axis = "X axis"
            
        axis = test_list[1]
#         print(axis)
#         print(test_list)
        if main_axis == "X axis":
            for coordinate in test_list:
                if coordinate[0] != axis[0]:
                    axis = coordinate
        elif main_axis == "Y axis":
            for coordinate in test_list:
                if coordinate[1] != axis[1]:
                    axis = coordinate
                
        return axis[0],axis[1],main_axis
    
   
    
    def y_axis_solution(self,df,data_row_num,data_col_num,true_place):
        
        table = df.iloc[data_row_num:, data_col_num:].reset_index(drop=True)  # <--- all useful data
        date = df.iloc[data_row_num:,data_col_num].astype(str)
        date = testTableParser.parse(date)
        table.iloc[0:,0] = date[0].reset_index(drop=True)
        index_of_table = list(table.columns)
#         print(index_of_table)
        num = index_of_table[0]
        index_of_table.pop(0)
        index_of_table.insert(true_place,num)
        table = table[index_of_table]
        min_height = 0
        if data_row_num > 3:min_height = data_row_num- 3
        elif data_row_num< 3 and data_row_num >1: min_height = data_row_num -1
        list_index =[]

        for i in range(data_col_num,df.shape[1]):
            buffer = ""
            if pd.isnull(df.iloc[min_height,i]) and len(list_index)!= 0:
        #         print(df.iloc[min_height,i])
                buffer = "upWord_"
            for j in range(min_height,data_row_num):
                buffer += str(df.iloc[j,i]).replace('\n',' ')+ "__"
                
            if buffer.strip().replace("nan__","") == "upWord_":
                list_index.append("-----")
            else: list_index.append(buffer.replace("nan__","").strip())
        #             list_index.append(buffer.replace("nan__","")[:-2])
        #     print(list_index)
            if list_index[-1][:7] == "upWord_":
                if list_index[-2] == "-----":
                    index = list_index[-3].find("_")
                    list_index[-1] = list_index[-3][:index+2]+list_index[-1][7:]
                else:
#                     print(list_index[-2],list_index[-2].count("__"))
                    index = list_index[-2].find("_")
                    list_index[-1] = list_index[-2][:index+2]+list_index[-1][7:]
        list_index = pd.Series(list_index).str[:-2]
        index_of_table.remove(0)
        index_of_table.insert(true_place,0)
        list_index = list_index.reindex(index_of_table)
        table.columns = list_index 
#         list_index.head(15)
        return table
    
    def x_axis_solution(self,df,data_row_num,data_col_num,true_place):
#         print(data_row_num,data_col_num)

        table = df.iloc[data_row_num:, data_col_num:].reset_index(drop=True)  # <--- all useful data
        date = df.iloc[data_row_num, data_col_num:].astype(str)  # <--- Series with date (01.03.2016;...)
        date = testTableParser.parse(date)
        table.iloc[0] = date[0]
        index_of_table = list(table.index)
        index_of_table.remove(0)
        index_of_table.insert(true_place,0)
        table = table.reindex(index_of_table)
#         print(list(table.index))
        min_deep = 0
        if data_col_num > 3:min_deep = data_col_num - 3
        elif data_col_num< 3 and data_col_num >1: min_deep = data_col_num -1
        list_index =[]
        for i in range(data_row_num,df.shape[0]):
            buffer = ""
            if pd.isnull(df.iloc[i,min_deep]) and len(list_index)!= 0:
                buffer = "upWord_"
            for j in range(min_deep,data_col_num):
                buffer +=str(df.iloc[i,j]).replace('\n',' ')+"__"
                
            if buffer.strip().replace("nan__","") == "upWord_":
                list_index.append("-----")
            else: list_index.append(buffer.replace("nan__","").strip()) 
            if list_index[-1][:7] == "upWord_":
                if list_index[-2] == "-----":
                    index = list_index[-3].find("_")
                    list_index[-1] = list_index[-3][:index+2]+list_index[-1][7:]
                else:
                    index = list_index[-2].find("_")
                    list_index[-1] = list_index[-2][:index+2]+list_index[-1][7:]
        list_index = pd.Series(list_index).str[:-2]
        
        list_index = list_index.reindex(index_of_table)
        table = table.set_index(list_index)
        return table
    
    def all_action(self,df): # <-----    drive method
        answer_list = []
        data_row_num, data_col_num,main_axis = self.find_coordinate(0,0,df) # <-- find coordinate of dates
#         print(data_row_num, data_col_num,main_axis)
        if main_axis == "X axis":
            index = list(df.index)
#             print(index)
#             print(data_row_num)
            index.remove(data_row_num)
            index.insert(0,data_row_num)
            df = df.reindex(index)
            true_place = data_row_num
            data_row_num, data_col_num,main_axis = self.find_coordinate(0,0,df) # <-- find coordinate of dates

#             print(data_row_num, data_col_num,main_axis)
        elif main_axis == "Y axis":
            divide_line = self.find_break(df,data_row_num,data_col_num,main_axis)
#             print(divide_line)
            if divide_line != 0:
                df_list  = [df.iloc[:divide_line,:].reset_index(drop=True),df.iloc[divide_line:,:].reset_index(drop=True)]
                df1,df2 = df_list[0],df_list[1]
                index1 = list(df1.columns)
                index1.remove(data_col_num)
                index1.insert(0,data_col_num)
                df1 = df1[index1]
                index2 = list(df2.columns)
                index2.remove(data_col_num)
                index2.insert(0,data_col_num)
                df2 = df2[index2]
                true_place = data_col_num
                data_row_num1, data_col_num1,main_axis = self.find_coordinate(0,0,df1) # <-- find coordinate of dates
                data_row_num2, data_col_num2,main_axis = self.find_coordinate(0,0,df2) # <-- find coordinate of dates
                answer_list = [self.y_axis_solution(df1,data_row_num1,data_col_num1,true_place),self.y_axis_solution(df2,data_row_num2,data_col_num2,true_place)]
                return answer_list
            else:
                index = list(df.columns)
                index.remove(data_col_num)
                index.insert(0,data_col_num)
                df = df[index]
                true_place = data_col_num
                data_row_num, data_col_num,main_axis = self.find_coordinate(0,0,df) # <-- find coordinate of dates
                return self.y_axis_solution(df,data_row_num,data_col_num,true_place)

                
        if main_axis == "X axis":
            return self.x_axis_solution(df,data_row_num,data_col_num,true_place)
#         elif main_axis == "Y axis":
#             return self.y_axis_solution(df,data_row_num,data_col_num,true_place)
        return df.sort_index(ascending=True)

In [70]:
testTableParser = TestTableParser()
# testTableParser.fix_xlsx('file1.xlsx')
# df = pd.read_excel("file22.xlsx",header=None)
# df = pd.read_csv("file44.csv",encoding='unicode_escape',header=None)

# xlsx = pd.ExcelFile('file22.xlsx')
# names = xlsx.sheet_names
# sheet_to_df_map = {}
# for i  in range(len(names)):
#     sheet_to_df_map[names[i]] = testTableParser.all_action(pd.read_excel(open('file22.xlsx', 'rb'),header=None,sheet_name=names[i]))
    
df = pd.read_excel(open('file22.xlsx', 'rb'),header=None,sheet_name='Стадо 1 ')
testTableParser.all_action(df)[1]

,Воз-ст 1 партии,Воз-ст 2 партии,Показатель__№ корпуса__Дата,1 корпус__Поголовье__кур,1 корпус__пет,1 корпус__Выбраковка__кур,1 корпус__пет,1 корпус__Ср вес 1 гол__кур,1 корпус__пет,1 корпус__Расход корма,...,15 корпус__Выбраковка__кур,15 корпус__пет,15 корпус__Ср вес 1 гол__кур,15 корпус__пет,15 корпус__Расход корма,15 корпус__Продуктивность,15 корпус__Вес яйца,15 корпус__Вода,"15 корпус__t, С утро",15 корпус__Влажность
0,137,134,2020-05-13 00:00:00+00:00,0,445,NaN,NaN,NaN,3241,44.055,...,NaN,NaN,NaN,3172,0,NaN,NaN,NaN,NaN,NaN
1,138,135,2020-05-14 00:00:00+00:00,0,445,NaN,NaN,NaN,NaN,44.055,...,NaN,NaN,NaN,NaN,96,NaN,NaN,NaN,NaN,NaN
2,139,136,2020-05-15 00:00:00+00:00,0,445,NaN,NaN,NaN,NaN,44.055,...,NaN,NaN,NaN,NaN,96,NaN,NaN,NaN,NaN,NaN
3,140,137,2020-05-16 00:00:00+00:00,0,445,NaN,NaN,NaN,NaN,44.055,...,NaN,NaN,NaN,NaN,96,NaN,NaN,NaN,NaN,NaN
4,141,138,2020-05-17 00:00:00+00:00,0,445,NaN,NaN,NaN,NaN,44.055,...,NaN,NaN,NaN,NaN,96,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
362,499,496,2021-05-10 00:00:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
363,500,497,2021-05-11 00:00:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
364,501,498,2021-05-12 00:00:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
365,502,499,2021-05-13 00:00:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [71]:
sheet_to_df_map.keys()

dict_keys(['Стадо 1 ', 'Стадо 2  ', 'Стадо 3 ', 'Стадо 4 ', 'Стадо 5 ', 'Стадо 6', 'Стадо 7', 'Стадо 8', 'Стадо 9', 'Стадо 10 ', 'Стадо 11', 'Стадо 12', 'Стадо 13'])

In [40]:
sheet_to_df_map['Стадо 1 ']

,Воз-ст 1 партии,Воз-ст 2 партии,Показатель__№ корпуса__Дата,1 корпус__Поголовье__кур,1 корпус__пет,1 корпус__Выбраковка__кур,1 корпус__пет,1 корпус__Ср вес 1 гол__кур,1 корпус__пет,1 корпус__Расход корма,...,15 корпус__Выбраковка__кур,15 корпус__пет,15 корпус__Ср вес 1 гол__кур,15 корпус__пет,15 корпус__Расход корма,15 корпус__Продуктивность,15 корпус__Вес яйца,15 корпус__Вода,"15 корпус__t, С утро",15 корпус__Влажность
0,137,134,2020-05-13 00:00:00+00:00,0,445,NaN,NaN,NaN,3241,44.055,...,NaN,NaN,NaN,3172,0,NaN,NaN,NaN,NaN,NaN
1,138,135,2020-05-14 00:00:00+00:00,0,445,NaN,NaN,NaN,NaN,44.055,...,NaN,NaN,NaN,NaN,96,NaN,NaN,NaN,NaN,NaN
2,139,136,2020-05-15 00:00:00+00:00,0,445,NaN,NaN,NaN,NaN,44.055,...,NaN,NaN,NaN,NaN,96,NaN,NaN,NaN,NaN,NaN
3,140,137,2020-05-16 00:00:00+00:00,0,445,NaN,NaN,NaN,NaN,44.055,...,NaN,NaN,NaN,NaN,96,NaN,NaN,NaN,NaN,NaN
4,141,138,2020-05-17 00:00:00+00:00,0,445,NaN,NaN,NaN,NaN,44.055,...,NaN,NaN,NaN,NaN,96,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
362,499,496,2021-05-10 00:00:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
363,500,497,2021-05-11 00:00:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
364,501,498,2021-05-12 00:00:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
365,502,499,2021-05-13 00:00:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [36]:
testTableParser.parse(pd.Series("1762.08"))


(0   1762-08-01
 dtype: datetime64[ns],
 [])

In [710]:
df_list  = [df.iloc[:157,:].reset_index(drop=True),df.iloc[157:,:].reset_index(drop=True)]
df_list[1].head(10)


,0,1,2,3,4,5,6,7,8,9,...,173,174,175,176,177,178,179,180,181,182
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,Стадо РМ №1,NaN,NaN,1 партия,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,Воз-ст 1 партии,Воз-ст 2 партии,Показатель,1 корпус,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,NaN,NaN,№ корпуса,Поголовье,NaN,Выбраковка,NaN,Ср вес 1 гол,NaN,Расход корма,...,Выбраковка,NaN,Ср вес 1 гол,NaN,Расход корма,Продуктивность,Вес яйца,Вода,"t, С утро",Влажность
7,NaN,NaN,Дата,кур,пет,кур,пет,кур,пет,NaN,...,кур,пет,кур,пет,NaN,NaN,NaN,NaN,NaN,NaN
8,137,134,2020-05-13 00:00:00,0,445,NaN,NaN,NaN,3241,44.055,...,NaN,NaN,NaN,3172,0,NaN,NaN,NaN,NaN,NaN
9,138,135,2020-05-14 00:00:00,0,445,NaN,NaN,NaN,NaN,44.055,...,NaN,NaN,NaN,NaN,96,NaN,NaN,NaN,NaN,NaN


In [48]:
string = "geeks_for_geeks" 

print(string.count("_"))

2


In [51]:
print(string.find("e",3))

11


In [2]:
print("as_asd".find("_"))

2


In [ ]:
df = pd.read_excel("file22.xlsx")
xls = pd.ExcelFile('file22.xlsx')
xls.sheet_names

In [ ]:
sheet_to_df_map = {}
for sheet_name in xls.sheet_names:
    sheet_to_df_map[sheet_name] = xls.parse(sheet_name)

In [ ]:
list(sheet_to_df_map.values())[0]

In [ ]:
type(df.iloc[3,2])

In [ ]:
type(pd.to_datetime('01.01.2021'))